<a href="https://colab.research.google.com/github/Ragulan-K/ML-Projects-24/blob/main/Py_snake_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pygame, random, sys
from pygame.locals import *
def collide(x1, x2, y1, y2, w1, w2, h1, h2):
	if x1+w1>x2 and x1<x2+w2 and y1+h1>y2 and y1<y2+h2:
		return True
	else:
		return False

def die(screen, score):
	f=pygame.font.SysFont('Arial', 30);t=f.render('Your score was: '+str(score), True, (0, 0, 0));screen.blit(t, (10, 270));pygame.display.update();pygame.time.wait(2000);sys.exit(0)

xs = [290, 290, 290, 290, 290];ys = [290, 270, 250, 230, 210];dirs = 0;score = 0;
applepos = (random.randint(0, 590), random.randint(0, 590));
pygame.init();

s=pygame.display.set_mode((600, 600));
pygame.display.set_caption('Snake');
appleimage = pygame.Surface((10, 10));
appleimage.fill((0, 255, 0));
img = pygame.Surface((20, 20));
img.fill((255, 0, 0));
f = pygame.font.SysFont('Arial', 20);
clock = pygame.time.Clock()
while True:
	clock.tick(10)
	for e in pygame.event.get():
		if e.type == QUIT:
			sys.exit(0)
		elif e.type == KEYDOWN:
			if e.key == K_UP and dirs != 0:dirs = 2
			elif e.key == K_DOWN and dirs != 2:dirs = 0
			elif e.key == K_LEFT and dirs != 1:dirs = 3
			elif e.key == K_RIGHT and dirs != 3:dirs = 1
	i = len(xs)-1
	while i >= 2:
		if collide(xs[0], xs[i], ys[0], ys[i], 20, 20, 20, 20):
			die(s, score)
		i-= 1
	if collide(xs[0], applepos[0], ys[0], applepos[1], 20, 10, 20, 10):
		score+=1;
		xs.append(700);
		ys.append(700);
		applepos=(random.randint(0,590),random.randint(0,590))
	print(xs,ys)

	if xs[0] < 0 or xs[0] > 580 or ys[0] < 0 or ys[0] > 580:
		die(s, score)
	i = len(xs)-1
	while i >= 1:
		xs[i] = xs[i-1];ys[i] = ys[i-1];i -= 1
	if dirs==0:ys[0] += 20
	elif dirs==1:xs[0] += 20
	elif dirs==2:ys[0] -= 20
	elif dirs==3:xs[0] -= 20
	s.fill((255, 255, 255))
	for i in range(0, len(xs)):
		s.blit(img, (xs[i], ys[i]))
	s.blit(appleimage, applepos);t=f.render(str(score), True, (0, 0, 0));s.blit(t, (10, 10));pygame.display.update()


##AI snake game

In [ ]:
from snake.game import Game, GameConf, GameMode

greedy = "GreedySolver"
hamilton = "HamiltonSolver"

normal = GameMode.NORMAL


conf = GameConf()
conf.solver_name = hamilton
conf.mode = normal
print("Solver: %s " % (conf.solver_name))
print("Mode: %s" %(conf.mode))
Game(conf).run()

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# pylint: disable=C0103,C0111,W0201,W0703

import errno
import os
import traceback
from enum import Enum, unique

from snake.base import Direc, Map, PointType, Pos, Snake
from snake.gui import GameWindow
from snake.solver import GreedySolver, HamiltonSolver


@unique
class GameMode(Enum):
    NORMAL = 0         # AI with GUI
    BENCHMARK = 1      # Run benchmarks without GUI
    TRAIN_DQN = 2      # Train DQNSolver without GUI
    TRAIN_DQN_GUI = 3  # Train DQNSolver with GUI


class GameConf:

    def __init__(self):
        """Initialize a default configuration."""

        # Game mode
        self.mode = GameMode.NORMAL

        # Solver
        self.solver_name = 'HamiltonSolver'  # Class name of the solver

        # Size
        self.map_rows = 8
        self.map_cols = self.map_rows
        self.map_width = 160  # pixels
        self.map_height = self.map_width
        self.info_panel_width = 155  # pixels
        self.window_width = self.map_width + self.info_panel_width
        self.window_height = self.map_height
        self.grid_pad_ratio = 0.25

        # Switch
        self.show_grid_line = False
        self.show_info_panel = True

        # Delay
        self.interval_draw = 50       # ms
        self.interval_draw_max = 200  # ms

        # Color
        self.color_bg = '#000000'
        self.color_txt = '#F5F5F5'
        self.color_line = '#424242'
        self.color_wall = '#F5F5F5'
        self.color_food = '#FFF59D'
        self.color_head = '#F5F5F5'
        self.color_body = '#F5F5F5'

        # Initial snake
        self.init_direc = Direc.RIGHT
        self.init_bodies = [Pos(1, 4), Pos(1, 3), Pos(1, 2), Pos(1, 1)]
        self.init_types = [PointType.HEAD_R] + [PointType.BODY_HOR] * 3

        # Font
        self.font_info = ('Arial', 9)

        # Info
        self.info_str = (
            "<w/a/s/d>: snake direction\n"
            "<space>: pause/resume\n"
            "<r>: restart    <esc>: exit\n"
            "-----------------------------------\n"
            "status: %s\n"
            "episode: %d   step: %d\n"
            "length: %d/%d (" + str(self.map_rows) + "x" + str(self.map_cols) + ")\n"
            "-----------------------------------"
        )
        self.info_status = ['eating', 'dead', 'full']

class Game:

    def __init__(self, conf):
        self._conf = conf
        self._map = Map(conf.map_rows + 2, conf.map_cols + 2)
        self._snake = Snake(self._map, conf.init_direc,
                            conf.init_bodies, conf.init_types)
        self._pause = False
        self._solver = globals()[self._conf.solver_name](self._snake)
        self._episode = 1
        self._init_log_file()

    @property
    def snake(self):
        return self._snake

    @property
    def episode(self):
        return self._episode

    def run(self):
        if self._conf.mode == GameMode.BENCHMARK:
            self._run_benchmarks()
        elif self._conf.mode == GameMode.TRAIN_DQN:
            self._run_dqn_train()
            self._plot_history()
        else:
            window = GameWindow("Snake", self._conf, self._map, self, self._on_exit, (
                ('<w>', lambda e: self._update_direc(Direc.UP)),
                ('<a>', lambda e: self._update_direc(Direc.LEFT)),
                ('<s>', lambda e: self._update_direc(Direc.DOWN)),
                ('<d>', lambda e: self._update_direc(Direc.RIGHT)),
                ('<r>', lambda e: self._reset()),
                ('<space>', lambda e: self._toggle_pause())
            ))
            if self._conf.mode == GameMode.NORMAL:
                window.show(self._game_main_normal)

    def _run_benchmarks(self):
        STEPS_LIMIT = 5000
        NUM_EPISODES = int(input("Please input the number of episodes: "))

        print("\nMap size: %dx%d" % (self._conf.map_rows, self._conf.map_cols))
        print("Solver: %s\n" % self._conf.solver_name[:-6].lower())

        tot_len, tot_steps = 0, 0

        for _ in range(NUM_EPISODES):
            print("Episode %d - " % self._episode, end="")
            while True:
                self._game_main_normal()
                if self._map.is_full():
                    print("FULL (len: %d | steps: %d)"
                          % (self._snake.len(), self._snake.steps))
                    break
                elif self._snake.dead:
                    print("DEAD (len: %d | steps: %d)"
                          % (self._snake.len(), self._snake.steps))
                    break
                elif self._snake.steps >= STEPS_LIMIT:
                    print("STEP LIMIT (len: %d | steps: %d)"
                          % (self._snake.len(), self._snake.steps))
                    self._write_logs()  # Write the last step
                    break
            tot_len += self._snake.len()
            tot_steps += self._snake.steps
            self._reset()

        avg_len = tot_len / NUM_EPISODES
        avg_steps = tot_steps / NUM_EPISODES
        print("\n[Summary]\nAverage Length: %.2f\nAverage Steps: %.2f\n"
              % (avg_len, avg_steps))

        self._on_exit()


    def _game_main_normal(self):
        if not self._map.has_food():
            self._map.create_rand_food()

        if self._pause or self._is_episode_end():
            return

        self._update_direc(self._solver.next_direc())

        if self._conf.mode == GameMode.NORMAL and self._snake.direc_next != Direc.NONE:
            self._write_logs()

        self._snake.move()

        if self._is_episode_end():
            self._write_logs()  # Write the last step

    def _plot_history(self):
        self._solver.plot()

    def _update_direc(self, new_direc):
        self._snake.direc_next = new_direc
        if self._pause:
            self._snake.move()

    def _toggle_pause(self):
        self._pause = not self._pause

    def _is_episode_end(self):
        return self._snake.dead or self._map.is_full()

    def _reset(self):
        self._snake.reset()
        self._episode += 1

    def _on_exit(self):
        if self._log_file:
            self._log_file.close()
        if self._solver:
            self._solver.close()

    def _init_log_file(self):
        try:
            os.makedirs("logs")
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise
        try:
            self._log_file = None
            self._log_file = open('logs/snake.log', 'w')
        except FileNotFoundError:
            if self._log_file:
                self._log_file.close()

    def _write_logs(self):
        self._log_file.write("[ Episode %d / Step %d ]\n" % \
                             (self._episode, self._snake.steps))
        for i in range(self._map.num_rows):
            for j in range(self._map.num_cols):
                pos = Pos(i, j)
                t = self._map.point(pos).type
                if t == PointType.EMPTY:
                    self._log_file.write("  ")
                elif t == PointType.WALL:
                    self._log_file.write("# ")
                elif t == PointType.FOOD:
                    self._log_file.write("F ")
                elif t == PointType.HEAD_L or t == PointType.HEAD_U or \
                    t == PointType.HEAD_R or t == PointType.HEAD_D:
                    self._log_file.write("H ")
                elif pos == self._snake.tail():
                    self._log_file.write("T ")
                else:
                    self._log_file.write("B ")
            self._log_file.write("\n")
        self._log_file.write("[ last/next direc: %s/%s ]\n" % \
                              (self._snake.direc, self._snake.direc_next))
        self._log_file.write("\n")


##GUI

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# pylint: disable=C0103,C0111

import tkinter as tk
from snake.base import Pos, PointType


class GameWindow(tk.Tk):

    def __init__(self, title, conf, game_map, game=None, on_exit=None, keybindings=None):
        super().__init__()
        super().title(title)
        super().resizable(width=False, height=False)
        super().configure(background=conf.color_bg)
        if conf.show_info_panel:
            super().geometry("%dx%d" % (conf.window_width, conf.window_height))

        self._conf = conf
        self._map = game_map
        self._grid_width = conf.map_width / (game_map.num_rows - 2)
        self._grid_height = conf.map_height / (game_map.num_cols - 2)
        self._init_widgets()
        self._init_draw_params()

        if game is not None:
            self._game = game
            self._snake = game.snake

        def on_destroy():
            if callable(on_exit):
                on_exit()
            self.destroy()

        self._init_keybindings(keybindings, on_destroy)

    def show(self, game_loop=None):
        def cb():
            if callable(game_loop):
                game_loop()
            self._update_contents()
            self.after(self._conf.interval_draw, cb)
        self.after(100, cb)
        self.mainloop()

    def _init_widgets(self):
        self._canvas = tk.Canvas(self,
                                 bg=self._conf.color_bg,
                                 width=self._conf.map_width,
                                 height=self._conf.map_height,
                                 highlightthickness=0)
        self._canvas.pack(side=tk.LEFT)

        if self._conf.show_info_panel:

            self._info_var = tk.StringVar()

            frm = tk.Frame(self, bg=self._conf.color_bg)
            frm.pack(side=tk.RIGHT, anchor=tk.N)

            tk.Message(frm,
                       textvariable=self._info_var,
                       fg=self._conf.color_txt,
                       bg=self._conf.color_bg,
                       font=self._conf.font_info).pack(side=tk.TOP, anchor=tk.W)

            scale = tk.Scale(frm,
                             font=self._conf.font_info,
                             fg=self._conf.color_txt,
                             bg=self._conf.color_bg,
                             highlightthickness=0,
                             from_=self._conf.interval_draw_max,
                             to=0,
                             orient=tk.HORIZONTAL,
                             length=self._conf.window_width - self._conf.map_width,
                             showvalue=False,
                             tickinterval=0,
                             resolution=1,
                             command=self._update_speed)
            scale.pack(side=tk.TOP, anchor=tk.W)
            scale.set(self._conf.interval_draw)

    def _update_speed(self, speed):
        self._conf.interval_draw = int(speed)

    def _init_keybindings(self, keybindings, on_destroy):
        self.bind('<Escape>', lambda e: on_destroy())
        self.protocol('WM_DELETE_WINDOW', on_destroy)
        if keybindings:
            for kb in keybindings:
                self.bind(kb[0], kb[1])

    def _init_draw_params(self):
        pad_ratio = self._conf.grid_pad_ratio
        food_pad_ratio = 0.9 * pad_ratio
        self._dx1 = pad_ratio * self._grid_width
        self._dx2 = (1 - pad_ratio) * self._grid_width + 1
        self._dy1 = pad_ratio * self._grid_height
        self._dy2 = (1 - pad_ratio) * self._grid_height + 1
        self._dx1_food = food_pad_ratio * self._grid_width
        self._dx2_food = (1 - food_pad_ratio) * self._grid_width
        self._dy1_food = food_pad_ratio * self._grid_height
        self._dy2_food = (1 - food_pad_ratio) * self._grid_height

    def _update_contents(self):
        self._canvas.delete(tk.ALL)
        self._draw_bg()
        if self._conf.show_grid_line:
            self._draw_grid_line()
        if self._conf.show_info_panel:
            self._draw_info_panel()
        self._draw_map_contents()
        self.update()

    def _draw_bg(self):
        self._canvas.create_rectangle(0, 0, self._conf.map_width, self._conf.map_height,
                                      fill=self._conf.color_bg, outline='')

    def _draw_grid_line(self):
        for i in range(1, self._map.num_rows - 2):
            for j in range(1, self._map.num_cols - 2):
                x = j * self._grid_width
                y = i * self._grid_height
                self._canvas.create_line(x, 0,
                                         x, self._conf.map_height,
                                         fill=self._conf.color_line)
                self._canvas.create_line(0, y,
                                         self._conf.map_width, y,
                                         fill=self._conf.color_line)

    def _draw_info_panel(self):
        self._canvas.create_line(self._conf.map_width - 1, 0,
                                 self._conf.map_width - 1, self._conf.map_height,
                                 fill=self._conf.color_line)

        if self._snake.dead:
            status_str = self._conf.info_status[1]
        elif self._map.is_full():
            status_str = self._conf.info_status[2]
        else:
            status_str = self._conf.info_status[0]

        self._info_var.set(self._conf.info_str %
                            (status_str,
                             self._game.episode, self._snake.steps,
                             self._snake.len(), self._map.capacity))

    def _draw_map_contents(self):
        for i in range(self._map.num_rows - 2):
            for j in range(self._map.num_cols - 2):
                self._draw_grid(j * self._grid_width, i * self._grid_height,
                                self._map.point(Pos(i + 1, j + 1)).type)

    def _draw_grid(self, x, y, t):
        if t == PointType.WALL:
            self._canvas.create_rectangle(x, y,
                                          x + self._grid_width, y + self._grid_height,
                                          fill=self._conf.color_wall, outline='')
        elif t == PointType.FOOD:
            self._canvas.create_rectangle(x + self._dx1_food, y + self._dy1_food,
                                          x + self._dx2_food, y + self._dy2_food,
                                          fill=self._conf.color_food, outline='')
        elif t == PointType.HEAD_L:
            self._canvas.create_rectangle(x + self._dx1, y + self._dy1,
                                          x + self._grid_width, y + self._dy2,
                                          fill=self._conf.color_head, outline='')
        elif t == PointType.HEAD_U:
            self._canvas.create_rectangle(x + self._dx1, y + self._dy1,
                                          x + self._dx2, y + self._grid_height,
                                          fill=self._conf.color_head, outline='')
        elif t == PointType.HEAD_R:
            self._canvas.create_rectangle(x, y + self._dy1,
                                          x + self._dx2, y + self._dy2,
                                          fill=self._conf.color_head, outline='')
        elif t == PointType.HEAD_D:
            self._canvas.create_rectangle(x + self._dx1, y,
                                          x + self._dx2, y + self._dy2,
                                          fill=self._conf.color_head, outline='')
        elif t == PointType.BODY_LU:
            self._canvas.create_rectangle(x, y + self._dy1,
                                          x + self._dx1, y + self._dy2,
                                          fill=self._conf.color_body, outline='')
            self._canvas.create_rectangle(x + self._dx1, y,
                                          x + self._dx2, y + self._dy2,
                                          fill=self._conf.color_body, outline='')
        elif t == PointType.BODY_UR:
            self._canvas.create_rectangle(x + self._dx1, y,
                                          x + self._dx2, y + self._dy2,
                                          fill=self._conf.color_body, outline='')
            self._canvas.create_rectangle(x + self._dx1, y + self._dy1,
                                          x + self._grid_width, y + self._dy2,
                                          fill=self._conf.color_body, outline='')
        elif t == PointType.BODY_RD:
            self._canvas.create_rectangle(x + self._dx1, y + self._dy1,
                                          x + self._grid_width, y + self._dy2,
                                          fill=self._conf.color_body, outline='')
            self._canvas.create_rectangle(x + self._dx1, y + self._dy1,
                                          x + self._dx2, y + self._grid_height,
                                          fill=self._conf.color_body, outline='')
        elif t == PointType.BODY_DL:
            self._canvas.create_rectangle(x + self._dx1, y + self._dy1,
                                          x + self._dx2, y + self._grid_height,
                                          fill=self._conf.color_body, outline='')
            self._canvas.create_rectangle(x, y + self._dy1,
                                          x + self._dx1, y + self._dy2,
                                          fill=self._conf.color_body, outline='')
        elif t == PointType.BODY_HOR:
            self._canvas.create_rectangle(x, y + self._dy1,
                                          x + self._grid_width, y + self._dy2,
                                          fill=self._conf.color_body, outline='')
        elif t == PointType.BODY_VER:
            self._canvas.create_rectangle(x + self._dx1, y,
                                          x + self._dx2, y + self._grid_height,
                                          fill=self._conf.color_body, outline='')


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# pylint: disable=C0103,C0111,W0201,W0212

"""Definition of class Snake."""

import random
from collections import deque

from snake.base.direc import Direc
from snake.base.point import PointType
from snake.base.pos import Pos


class Snake:
    """Snake of the game."""

    def __init__(self, game_map, init_direc=None, init_bodies=None, init_types=None):
        """Initialize a Snake object.

        Args:
            game_map (base.map.Map): The map that the snake moves on.
            init_direc (base.direc.Direc): Initial direction.
            init_bodies (list of base.pos.Pos): Initial snake bodies positions.
            init_types (list of base.point.PointType): Types of each position in init_bodies.

        """
        self._map = game_map
        self._init_direc = init_direc
        self._init_bodies = init_bodies
        self._init_types = init_types
        self.reset(False)

    def reset(self, reset_map=True):
        rand_init = False
        if self._init_direc is None:  # Randomly initialize
            rand_init = True
            head_row = random.randrange(2, self._map.num_rows - 2)
            head_col = random.randrange(2, self._map.num_cols - 2)
            head = Pos(head_row, head_col)

            self._init_direc = random.choice([Direc.LEFT, Direc.UP, Direc.RIGHT, Direc.DOWN])
            self._init_bodies = [head, head.adj(Direc.opposite(self._init_direc))]

            self._init_types = []
            if self._init_direc == Direc.LEFT:
                self._init_types.append(PointType.HEAD_L)
            elif self._init_direc == Direc.UP:
                self._init_types.append(PointType.HEAD_U)
            elif self._init_direc == Direc.RIGHT:
                self._init_types.append(PointType.HEAD_R)
            elif self._init_direc == Direc.DOWN:
                self._init_types.append(PointType.HEAD_D)
            if self._init_direc == Direc.LEFT or self._init_direc == Direc.RIGHT:
                self._init_types.append(PointType.BODY_HOR)
            elif self._init_direc == Direc.UP or self._init_direc == Direc.DOWN:
                self._init_types.append(PointType.BODY_VER)

        self._steps = 0
        self._dead = False
        self._direc = self._init_direc
        self._direc_next = Direc.NONE
        self._bodies = deque(self._init_bodies)

        if reset_map:
            self._map.reset()
        for i, pos in enumerate(self._init_bodies):
            self._map.point(pos).type = self._init_types[i]

        if rand_init:
            self._init_direc = self._init_bodies = self._init_types = None

    def copy(self):
        m_copy = self._map.copy()
        s_copy = Snake(m_copy, Direc.NONE, [], [])
        s_copy._steps = self._steps
        s_copy._dead = self._dead
        s_copy._direc = self._direc
        s_copy._direc_next = self._direc_next
        s_copy._bodies = deque(self._bodies)
        return s_copy, m_copy

    @property
    def map(self):
        return self._map

    @property
    def steps(self):
        return self._steps

    @property
    def dead(self):
        return self._dead

    @dead.setter
    def dead(self, val):
        self._dead = val

    @property
    def direc(self):
        return self._direc

    @property
    def direc_next(self):
        return self._direc_next

    @direc_next.setter
    def direc_next(self, val):
        self._direc_next = val

    @property
    def bodies(self):
        return self._bodies

    def len(self):
        return len(self._bodies)

    def head(self):
        if not self._bodies:
            return None
        return self._bodies[0]

    def tail(self):
        if not self._bodies:
            return None
        return self._bodies[-1]

    def move_path(self, path):
        for p in path:
            self.move(p)

    def move(self, new_direc=None):
        if new_direc is not None:
            self._direc_next = new_direc

        if self._dead or \
           self._direc_next == Direc.NONE or \
           self._map.is_full() or \
           self._direc_next == Direc.opposite(self._direc):
            return

        old_head_type, new_head_type = self._new_types()
        self._map.point(self.head()).type = old_head_type
        new_head = self.head().adj(self._direc_next)
        self._bodies.appendleft(new_head)

        if not self._map.is_safe(new_head):
            self._dead = True
        if self._map.point(new_head).type == PointType.FOOD:
            self._map.rm_food()
        else:
            self._rm_tail()

        self._map.point(new_head).type = new_head_type
        self._direc = self._direc_next
        self._steps += 1

    def _rm_tail(self):
        self._map.point(self.tail()).type = PointType.EMPTY
        self._bodies.pop()

    def _new_types(self):
        old_head_type, new_head_type = None, None
        # new_head_type
        if self._direc_next == Direc.LEFT:
            new_head_type = PointType.HEAD_L
        elif self._direc_next == Direc.UP:
            new_head_type = PointType.HEAD_U
        elif self._direc_next == Direc.RIGHT:
            new_head_type = PointType.HEAD_R
        elif self._direc_next == Direc.DOWN:
            new_head_type = PointType.HEAD_D
        # old_head_type
        if (self._direc == Direc.LEFT and self._direc_next == Direc.LEFT) or \
           (self._direc == Direc.RIGHT and self._direc_next == Direc.RIGHT):
            old_head_type = PointType.BODY_HOR
        elif (self._direc == Direc.UP and self._direc_next == Direc.UP) or \
             (self._direc == Direc.DOWN and self._direc_next == Direc.DOWN):
            old_head_type = PointType.BODY_VER
        elif (self._direc == Direc.RIGHT and self._direc_next == Direc.UP) or \
             (self._direc == Direc.DOWN and self._direc_next == Direc.LEFT):
            old_head_type = PointType.BODY_LU
        elif (self._direc == Direc.LEFT and self._direc_next == Direc.UP) or \
             (self._direc == Direc.DOWN and self._direc_next == Direc.RIGHT):
            old_head_type = PointType.BODY_UR
        elif (self._direc == Direc.LEFT and self._direc_next == Direc.DOWN) or \
             (self._direc == Direc.UP and self._direc_next == Direc.RIGHT):
            old_head_type = PointType.BODY_RD
        elif (self._direc == Direc.RIGHT and self._direc_next == Direc.DOWN) or \
             (self._direc == Direc.UP and self._direc_next == Direc.LEFT):
            old_head_type = PointType.BODY_DL
        return old_head_type, new_head_type


##Position

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# pylint: disable=C0103,C0111

"""Definitions of class Pos."""

from snake.base.direc import Direc


class Pos:
    """Integer coordinate in 2D plane.

    The origin of the coordinate system is at the top-left corner,
    with x-axis extends downward and y-axis extends rightward.

    """
    def __init__(self, x=0, y=0):
        self._x = x
        self._y = y

    def __str__(self):
        return 'Pos(%d,%d)' % (self._x, self._y)
    __repr__ = __str__

    def __eq__(self, other):
        if isinstance(self, other.__class__):
            return self._x == other.x and self._y == other.y
        return NotImplemented

    def __pos__(self):
        return Pos(self._x, self._y)

    def __neg__(self):
        return Pos(-self._x, -self._y)

    def __add__(self, other):
        if isinstance(self, other.__class__):
            return Pos(self._x + other.x, self._y + other.y)
        return NotImplemented

    def __sub__(self, other):
        if isinstance(self, other.__class__):
            return self + (-other)
        return NotImplemented

    def __hash__(self):
        return hash((self.x, self.y))

    @staticmethod
    def manhattan_dist(p1, p2):
        return abs(p1.x - p2.x) + abs(p1.y - p2.y)

    def direc_to(self, adj_pos):
        """Return the direction of an adjacent Pos relative to self."""
        if self._x == adj_pos.x:
            diff = self._y - adj_pos.y
            if diff == 1:
                return Direc.LEFT
            elif diff == -1:
                return Direc.RIGHT
        elif self._y == adj_pos.y:
            diff = self._x - adj_pos.x
            if diff == 1:
                return Direc.UP
            elif diff == -1:
                return Direc.DOWN
        return Direc.NONE

    def adj(self, direc):
        """Return the adjacent Pos in a given direction."""
        if direc == Direc.LEFT:
            return Pos(self._x, self._y - 1)
        elif direc == Direc.RIGHT:
            return Pos(self._x, self._y + 1)
        elif direc == Direc.UP:
            return Pos(self._x - 1, self._y)
        elif direc == Direc.DOWN:
            return Pos(self._x + 1, self._y)
        else:
            return None

    def all_adj(self):
        """Return a list of all the adjacent Pos."""
        adjs = []
        for direc in Direc:
            if direc != Direc.NONE:
                adjs.append(self.adj(direc))
        return adjs

    @property
    def x(self):
        return self._x

    @x.setter
    def x(self, val):
        self._x = val

    @property
    def y(self):
        return self._y

    @y.setter
    def y(self, val):
        self._y = val


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# pylint: disable=C0103,C0111

"""Definition of class Point."""

from enum import Enum, unique


@unique
class PointType(Enum):
    """Type of the points on the game map."""
    EMPTY = 0
    WALL = 1
    FOOD = 2
    HEAD_L = 100
    HEAD_U = 101
    HEAD_R = 102
    HEAD_D = 103
    BODY_LU = 104
    BODY_UR = 105
    BODY_RD = 106
    BODY_DL = 107
    BODY_HOR = 108
    BODY_VER = 109


class Point:
    """Point on the game map."""
    def __init__(self):
        self._type = PointType.EMPTY

    @property
    def type(self):
        return self._type

    @type.setter
    def type(self, val):
        self._type = val


##Map

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# pylint: disable=C0103,C0111,W0201,W0212

"""Definition of class Map."""

import random

from snake.base.point import Point, PointType
from snake.base.pos import Pos


class Map:
    """2D game map."""

    def __init__(self, num_rows, num_cols):
        """Initialize a Map object."""
        if not isinstance(num_rows, int) or not isinstance(num_cols, int):
            raise TypeError("\'num_rows\' and \'num_cols\' must be integers")
        if num_rows < 5 or num_cols < 5:
            raise ValueError("\'num_rows\' and \'num_cols\' must >= 5")

        self._num_rows = num_rows
        self._num_cols = num_cols
        self._capacity = (num_rows - 2) * (num_cols - 2)
        self._content = [[Point() for _ in range(num_cols)] for _ in range(num_rows)]
        self.reset()

    def reset(self):
        self._food = None
        for i in range(self._num_rows):
            for j in range(self._num_cols):
                if i == 0 or i == self._num_rows - 1 or \
                   j == 0 or j == self._num_cols - 1:
                    self._content[i][j].type = PointType.WALL
                else:
                    self._content[i][j].type = PointType.EMPTY

    def copy(self):
        m_copy = Map(self._num_rows, self._num_cols)
        for i in range(self._num_rows):
            for j in range(self._num_cols):
                m_copy._content[i][j].type = self._content[i][j].type
        return m_copy

    @property
    def num_rows(self):
        return self._num_rows

    @property
    def num_cols(self):
        return self._num_cols

    @property
    def capacity(self):
        return self._capacity

    @property
    def food(self):
        return self._food

    def point(self, pos):
        """Return a point on the map.

        DO NOT directly modify the point type to PointType.FOOD and vice versa.
        Use {add|rm}_food() methods instead.

        Args:
            pos (base.pos.Pos): The position of the point to be fetched

        Returns:
            snake.point.Point: The point at the given position.

        """
        return self._content[pos.x][pos.y]

    def is_inside(self, pos):
        return pos.x > 0 and pos.x < self.num_rows - 1 \
               and pos.y > 0 and pos.y < self.num_cols - 1

    def is_empty(self, pos):
        return self.is_inside(pos) and self.point(pos).type == PointType.EMPTY

    def is_safe(self, pos):
        return self.is_inside(pos) and (self.point(pos).type == PointType.EMPTY or \
                                        self.point(pos).type == PointType.FOOD)

    def is_full(self):
        """Check if the map is filled with the snake's bodies."""
        for i in range(1, self.num_rows - 1):
            for j in range(1, self.num_cols - 1):
                t = self._content[i][j].type
                if t.value < PointType.HEAD_L.value:
                    return False
        return True

    def has_food(self):
        return self._food is not None

    def rm_food(self):
        if self.has_food():
            self.point(self._food).type = PointType.EMPTY
            self._food = None

    def create_food(self, pos):
        self.point(pos).type = PointType.FOOD
        self._food = pos
        return self._food

    def create_rand_food(self):
        empty_pos = []
        for i in range(1, self._num_rows - 1):
            for j in range(1, self._num_cols - 1):
                t = self._content[i][j].type
                if t == PointType.EMPTY:
                    empty_pos.append(Pos(i, j))
                elif t == PointType.FOOD:
                    return None  # Stop if food exists
        if empty_pos:
            return self.create_food(random.choice(empty_pos))
        else:
            return None


##Dir

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# pylint: disable=C0103,C0111

"""Definition of enum Direc."""

from enum import Enum, unique


@unique
class Direc(Enum):
    """Directions on the game plane."""
    NONE = 0
    LEFT = 1
    UP = 2
    RIGHT = 3
    DOWN = 4

    @staticmethod
    def opposite(direc):
        """Return the opposite direction."""
        if direc == Direc.LEFT:
            return Direc.RIGHT
        elif direc == Direc.RIGHT:
            return Direc.LEFT
        elif direc == Direc.UP:
            return Direc.DOWN
        elif direc == Direc.DOWN:
            return Direc.UP
        else:
            return Direc.NONE


##Initiation

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""Basic elements of Snake."""

from snake.base.direc import Direc
from snake.base.pos import Pos
from snake.base.point import PointType, Point
from snake.base.map import Map
from snake.base.snake import Snake
